# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

### Imports

In [1]:
import os
import requests
from dotenv import load_dotenv

import pandas as pd
import polars as pl
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns


from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostClassifier, Pool

#import calendar
import joblib
#import s3fs
import gc
import boto3
from botocore.exceptions import ClientError



/home/mle-user/mle_projects/mle-project-sprint-4/.venv_recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Config

In [2]:
# Fix random state for reproducibility
RANDOM_STATE = 42

# Load environment variables
load_dotenv()

datasets = {
    'tracks.parquet': os.getenv('RAW_URL_TRACKS'),
    'catalog_names.parquet': os.getenv('RAW_URL_CATALOG_NAMES'),
    'interactions.parquet': os.getenv('RAW_URL_INTERACTIONS'),
}

raw_dir = os.getenv('RAW_DATA_DIR', '../data/raw')
preprocessed_dir = os.getenv('PREPROCESSED_DATA_DIR', '../data/preprocessed')
encoder_dir = os.getenv('ENCODERS_DIR', '../encoders')

s3_bucket = os.getenv('S3_BUCKET_NAME')
s3_prefix = os.getenv('S3_PREFIX', 'recsys/data/')
s3_region = os.getenv('S3_REGION', 'us-east-1')
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')


# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [3]:
# ---------- Download datasets and save locally ---------- #
# Create directory
os.makedirs(raw_dir, exist_ok=True)

# Download and save each dataset
for filename, url in datasets.items():
    save_path = os.path.join(raw_dir, filename)
    response = requests.get(url)
    response.raise_for_status()
    with open(save_path, 'wb') as f:
        f.write(response.content)
    print(f'Saved {filename} to {save_path}')

Saved tracks.parquet to data/raw/tracks.parquet
Saved catalog_names.parquet to data/raw/catalog_names.parquet
Saved interactions.parquet to data/raw/interactions.parquet


In [4]:
# ---------- Load datasets ---------- #
tracks = pl.read_parquet(os.path.join(raw_dir, 'tracks.parquet'))
catalog_names = pl.read_parquet(os.path.join(raw_dir, 'catalog_names.parquet'))
interactions = pl.read_parquet(os.path.join(raw_dir, 'interactions.parquet'))

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [5]:
# ---------- Check data summary ---------- #
def data_summary(df: pd.DataFrame, name: str):
    '''
        Display a quick overview of a DataFrame.
    '''

    print(f'\n===== {name.upper()} =====')  
  
    # Sample rows
    print('\nSample rows:')
    display(df.head())

    # Shape
    rows, cols = df.shape
    print(f'\nShape: {rows} rows x {cols} columns')
    
    # Data info
    print('\nSummary for numeric columns:')
    print(df.describe())
   
    # Unique values (column-wise, skip if error occurs)
    print('\nUnique values (for each column):')
    try:
        for col in df.columns:
            print(f'\nColumn: {col}')
            print(df[col].value_counts())
    except Exception as e:
        print(f'Skipped value_counts due to error: {e}')
    
    # Missing values
    print('\nMissing values:')
    print(df.null_count())

In [6]:
data_summary(tracks, 'tracks')


===== TRACKS =====

Sample rows:


track_id,albums,artists,genres
i64,list[i64],list[i64],list[i64]
26,"[3, 2490753]",[16],"[11, 21]"
38,"[3, 2490753]",[16],"[11, 21]"
135,"[12, 214, 2490809]",[84],[11]
136,"[12, 214, 2490809]",[84],[11]
138,"[12, 214, … 17294156]",[84],[11]



Shape: 1000000 rows x 4 columns

Summary for numeric columns:
shape: (9, 5)
┌────────────┬──────────────┬────────┬─────────┬────────┐
│ statistic  ┆ track_id     ┆ albums ┆ artists ┆ genres │
│ ---        ┆ ---          ┆ ---    ┆ ---     ┆ ---    │
│ str        ┆ f64          ┆ f64    ┆ f64     ┆ f64    │
╞════════════╪══════════════╪════════╪═════════╪════════╡
│ count      ┆ 1e6          ┆ 1e6    ┆ 1e6     ┆ 1e6    │
│ null_count ┆ 0.0          ┆ 0.0    ┆ 0.0     ┆ 0.0    │
│ mean       ┆ 3.6851e7     ┆ null   ┆ null    ┆ null   │
│ std        ┆ 2.6798e7     ┆ null   ┆ null    ┆ null   │
│ min        ┆ 26.0         ┆ null   ┆ null    ┆ null   │
│ 25%        ┆ 1.5430883e7  ┆ null   ┆ null    ┆ null   │
│ 50%        ┆ 3.4550746e7  ┆ null   ┆ null    ┆ null   │
│ 75%        ┆ 5.6925472e7  ┆ null   ┆ null    ┆ null   │
│ max        ┆ 1.01521819e8 ┆ null   ┆ null    ┆ null   │
└────────────┴──────────────┴────────┴─────────┴────────┘

Unique values (for each column):

Column: track_id
s

In [7]:
data_summary(catalog_names, 'catalog_names')


===== CATALOG_NAMES =====

Sample rows:


id,type,name
i64,str,str
3,"""album""","""Taller Children"""
12,"""album""","""Wild Young Hearts"""
13,"""album""","""Lonesome Crow"""
17,"""album""","""Graffiti Soul"""
26,"""album""","""Blues Six Pack"""



Shape: 1812471 rows x 3 columns

Summary for numeric columns:
shape: (9, 4)
┌────────────┬──────────────┬─────────┬─────────────────────────────────┐
│ statistic  ┆ id           ┆ type    ┆ name                            │
│ ---        ┆ ---          ┆ ---     ┆ ---                             │
│ str        ┆ f64          ┆ str     ┆ str                             │
╞════════════╪══════════════╪═════════╪═════════════════════════════════╡
│ count      ┆ 1.812471e6   ┆ 1812471 ┆ 1812471                         │
│ null_count ┆ 0.0          ┆ 0       ┆ 0                               │
│ mean       ┆ 2.3216e7     ┆ null    ┆ null                            │
│ std        ┆ 2.5260e7     ┆ null    ┆ null                            │
│ min        ┆ 0.0          ┆ album   ┆ 	Drake                          │
│ 25%        ┆ 3.480527e6   ┆ null    ┆ null                            │
│ 50%        ┆ 1.211436e7   ┆ null    ┆ null                            │
│ 75%        ┆ 3.773825e7   ┆ null 

In [8]:
data_summary(interactions, 'interactions')


===== INTERACTIONS =====

Sample rows:


user_id,track_id,track_seq,started_at,__index_level_0__
i32,i32,i16,datetime[ns],i64
0,99262,1,2022-07-17 00:00:00,0
0,589498,2,2022-07-19 00:00:00,1
0,590262,3,2022-07-21 00:00:00,2
0,590303,4,2022-07-22 00:00:00,3
0,590692,5,2022-07-22 00:00:00,4



Shape: 222629898 rows x 5 columns

Summary for numeric columns:
shape: (9, 6)
┌────────────┬───────────────┬──────────────┬──────────────┬───────────────────┬───────────────────┐
│ statistic  ┆ user_id       ┆ track_id     ┆ track_seq    ┆ started_at        ┆ __index_level_0__ │
│ ---        ┆ ---           ┆ ---          ┆ ---          ┆ ---               ┆ ---               │
│ str        ┆ f64           ┆ f64          ┆ f64          ┆ str               ┆ f64               │
╞════════════╪═══════════════╪══════════════╪══════════════╪═══════════════════╪═══════════════════╡
│ count      ┆ 2.22629898e8  ┆ 2.22629898e8 ┆ 2.22629898e8 ┆ 222629898         ┆ 2.22629898e8      │
│ null_count ┆ 0.0           ┆ 0.0          ┆ 0.0          ┆ 0                 ┆ 0.0               │
│ mean       ┆ 687576.660624 ┆ 3.6536e7     ┆ 462.140293   ┆ 2022-08-29        ┆ 461.140293        │
│            ┆               ┆              ┆              ┆ 16:39:44.541353   ┆                   │
│ std       

### Main takeaways
1. Tracs dataframe contains lists instead of scalar values. This can cause several problems:
- Unable to get insights on data;
Counting, merging, or joining on list columns is tricky.
- Missing values are hidden;
df.isna().sum() can’t detect empty lists, so there might be tracks with no genres or artists but they look as not missing values.
- Hard to work with for ML models.
Most algorithms expect scalar values, not lists.
Thus, it's necessary to explode the lists to get a dataframe with one row per track-per-item (artist, genre, album).

2. Catalog_names dataframe is in a format where everything (tracks, albums, artists, genres) is stacked in one column, and the type column tells what each row represents. This format isn't convenient for futher work. Thus, it's necessary to split catalog_names into several dataframes.